In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0) 

In [2]:

from langchain.prompts import ChatPromptTemplate
SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to make each wine selection and pairing a memorable experience for users."
wine_query = "이 와인에 어울리는 요리에는 어떤 것들이 있을까요?"

chat_template = ChatPromptTemplate.from_messages(
  [
    ('system', SYSTEM_PROMPT),
    ('human', [ {'type': 'text', 'text':'{text}'},
                {'type':'image_url', 'image_url':  {'url': '{image_url}'}}])
  ]
)
chat_template

ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to m

# recomment_dish_chain 용 랭체인 프롬프트

In [3]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = chat_template|llm|output_parser
response = chain.invoke(
  {
    'text': wine_query,
    'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
  }
)
response

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **그릴드 스테이크**: 육즙이 풍부한 스테이크는 Primitivo의 풍부한 과일 향과 잘 어울립니다.\n\n2. **바비큐 립**: 달콤하고 매콤한 바비큐 소스가 Primitivo의 스파이시한 노트와 조화를 이룹니다.\n\n3. **라자냐**: 토마토 소스와 치즈가 듬뿍 들어간 라자냐는 이 와인의 풍미를 잘 받쳐줍니다.\n\n4. **양고기 요리**: 허브와 함께 조리한 양고기는 Primitivo의 복합적인 맛과 잘 어울립니다.\n\n5. **치즈 플래터**: 특히 고다, 체다 같은 숙성 치즈와 잘 어울립니다.\n\n이 와인은 풍미가 강한 요리와 잘 어울리므로, 다양한 고기 요리나 풍부한 소스가 있는 요리와 함께 즐기시면 좋습니다.'

In [4]:
def recommend_dishes_chain(query):
    
  chat_template = ChatPromptTemplate.from_messages(
    [
      ('system', SYSTEM_PROMPT),
      ('human', [ {'type': 'text', 'text':query['text']},
                  {'type':'image_url', 'image_url':  {'url': query['image_url']}}])
    ]
  )
  chain = chat_template|llm|output_parser
  return chain

In [5]:
query_1 = {
  'text':wine_query,
  'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}
rec_dish_chain = recommend_dishes_chain(query_1)

In [6]:
rec_dish_chain.invoke(query_1)

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 종종 높은 알코올 도수를 가지고 있습니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **바비큐 요리**: 풍부한 소스와 함께한 바비큐 립이나 스테이크는 Primitivo의 과일 향과 잘 어울립니다.\n\n2. **토마토 소스 파스타**: 특히 미트볼이나 소시지가 들어간 파스타는 와인의 풍미를 잘 살려줍니다.\n\n3. **양고기**: 허브와 함께 구운 양고기는 Primitivo의 복합적인 맛과 잘 어울립니다.\n\n4. **치즈**: 고다, 체다 같은 강한 맛의 치즈와도 잘 어울립니다.\n\n5. **피자**: 특히 살라미나 페퍼로니가 올라간 피자는 Primitivo의 풍부한 맛과 조화를 이룹니다.\n\n이 와인은 강한 맛의 요리와 잘 어울리므로, 풍미가 강한 재료를 사용하는 요리를 선택하는 것이 좋습니다.'

In [7]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'Primitivo 와인은 풍부하고 과일 향이 강하며, 종종 높은 알코올 도수를 가지고 있습니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **바비큐 요리**: 풍부한 소스와 함께한 바비큐 립이나 스테이크는 Primitivo의 과일 향과 잘 어울립니다.\n\n2. **토마토 소스 파스타**: 토마토의 산미가 와인의 풍미를 잘 보완합니다.\n\n3. **양고기**: 허브와 함께 구운 양고기는 Primitivo의 깊은 맛과 잘 어울립니다.\n\n4. **치즈**: 고다나 체다 같은 숙성 치즈와도 잘 어울립니다.\n\n5. **피자**: 특히 고기 토핑이 있는 피자는 Primitivo와 좋은 조화를 이룹니다.\n\n이 와인의 풍부한 맛을 잘 살릴 수 있는 요리를 선택해 보세요!'

# describe_dish_flavor_chain 용 랭체인 프롬프트

In [8]:
def describe_dish_flavor_chain(query):
  #prompttemplate 
  # parser
  #chain
  return ''

In [9]:
dish_system_prompt = """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """

In [10]:
def describe_dish_flavor_chain(query):
    # 이미지가 있는 경우와 없는 경우를 구분하여 처리
    if query.get("image_urls"):
        # 이미지가 있는 경우: 멀티모달 프롬프트
        messages = [
            ("system", dish_system_prompt),
            ("human", [ #"human" 으로 해도 무방 - 역할 태그의 차이가 크지 않음
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        # 텍스트만 있는 경우 (이미지 없이는 요리 설명이 어려우므로 안내 메시지)
        messages = [
            ("system", dish_system_prompt),
            ("user", "요리 이미지가 제공되지 않았습니다. 요리 이미지를 업로드해주시면 정확한 분석을 도와드리겠습니다.")
        ]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser
    return chain

In [11]:
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke({
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)

이 요리는 견과류와 귀리가 들어간 그래놀라 바로, 고소하고 달콤하며 바삭한 식감이 특징입니다.


# wine_search

In [12]:

from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [13]:
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# 현재 계정의 모든 인덱스 목록 출력
print(pc.list_indexes())


[{
    "name": "quickstart",
    "metric": "cosine",
    "host": "quickstart-2wxeig8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}, {
    "name": "wine-embeddings",
    "metric": "cosine",
    "host": "wine-embeddings-2wxeig8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}, {
    "name": "embedding-3d",
    "metric": "cosine",
    "host": "embedding-3d-2wxeig8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
 

In [14]:
# from langchain_openai import OpenAIEmbeddings
# from langchain_pinecone import PineconeVectorStore

# embedding = OpenAIEmbeddings(model='text-embedding-3-small')
# vector_store = PineconeVectorStore(
#     index_name=os.getenv("wine-embeddings"),
#     embedding=embedding,
#     pinecone_api_key=os.getenv("PINECONE_API_KEY")
# )


import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

# Hugging Face 임베딩 모델 로드
embedding = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-mpnet-base-v2'
)

# Pinecone 벡터스토어 연결
vector_store = PineconeVectorStore(
    index_name="wine-embeddings",   # Pinecone에 생성한 인덱스 이름
    embedding=embedding,
    pinecone_api_key=os.getenv("PINECONE_API_KEY")
)


c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore
C:\Users\Admin

In [15]:

# vector store에서 현재 요리의 풍미와 맛과 유사한 와인 검색
taste_query =  "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
results = vector_store.similarity_search(
  taste_query, 
    k=5, 
    namespace=os.getenv("PINECONE_NAMESPACE")
)

results

[Document(id='82849', metadata={}, page_content=': 82849\ncountry: Croatia\ndescription: This has aromas of Bartlett pear and black Mission figs. In the mouth, there are flavors of pear, lemon and fresh picked apricots.\ndesignation: Vrhunsko Bijelo Vino\npoints: 88\nprice: 20.0\nprovince: Hvar\nregion_1: \nregion_2: \ntaster_name: Jeff Jenssen\ntaster_twitter_handle: @worldwineguys\ntitle: Zlatan Otok 2012 Vrhunsko Bijelo Vino Posip (Hvar)\nvariety: Posip\nwinery: Zlatan Otok'),
 Document(id='86421', metadata={}, page_content=": 86421\ncountry: Hungary\ndescription: Scents of honey and citrus rind on this golden Tokaji are marked by lifted apple and pear notes. Sweet tropical fruit and vanilla notes are pleasant, but there's a slightly tinned, apple juice concentrate flavor that persists on the palate.\ndesignation: Aszú 4 Puttonyos - 500 ml\npoints: 85\nprice: 30.0\nprovince: Tokaji\nregion_1: \nregion_2: \ntaster_name: Anna Lee C. Iijima\ntaster_twitter_handle: \ntitle: Tokaj Keresk

In [16]:
results = vector_store.similarity_search(
   response, 
    k=5, 
    namespace=os.getenv("PINECONE_NAMESPACE")
)

results

[Document(id='98628', metadata={}, page_content=": 98628\ncountry: Italy\ndescription: This aged expression of Motto Piane Soave drinks with such intensity, thickness and golden brilliance, you'd almost mistake it for a dessert wine. Aromas include candied fruit and sweet pineapple.\ndesignation: Motto Piane\npoints: 89\nprice: 18.0\nprovince: Veneto\nregion_1: Soave\nregion_2: \ntaster_name: \ntaster_twitter_handle: \ntitle: Fattori 2008 Motto Piane  (Soave)\nvariety: Garganega\nwinery: Fattori"),
 Document(id='95926', metadata={}, page_content=': 95926\ncountry: Portugal\ndescription: The Jaen grape, local to the Dão, has produced a perfumed juicy wine that is ripe and intensely full of acidity at the same time. It has only light tannins and red berry flavors that are right up front. The wine is ready to drink.\ndesignation: Enxertia\npoints: 87\nprice: 35.0\nprovince: Dão\nregion_1: \nregion_2: \ntaster_name: Roger Voss\ntaster_twitter_handle: @vossroger\ntitle: Casa da Passarella 2

In [17]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor, 
        k=5, 
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

In [18]:
runnable = RunnableLambda(search_wine)
response = runnable.invoke(response)
print(response['dish_flavor'])
print(response['wine_reviews'])

이 요리는 견과류와 귀리가 들어간 그래놀라 바로, 고소하고 달콤하며 바삭한 식감이 특징입니다.
: 98628
country: Italy
description: This aged expression of Motto Piane Soave drinks with such intensity, thickness and golden brilliance, you'd almost mistake it for a dessert wine. Aromas include candied fruit and sweet pineapple.
designation: Motto Piane
points: 89
price: 18.0
province: Veneto
region_1: Soave
region_2: 
taster_name: 
taster_twitter_handle: 
title: Fattori 2008 Motto Piane  (Soave)
variety: Garganega
winery: Fattori
: 95926
country: Portugal
description: The Jaen grape, local to the Dão, has produced a perfumed juicy wine that is ripe and intensely full of acidity at the same time. It has only light tannins and red berry flavors that are right up front. The wine is ready to drink.
designation: Enxertia
points: 87
price: 35.0
province: Dão
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Casa da Passarella 2012 Enxertia Jaen (Dão)
variety: Jaen
winery: Casa da Passarella
: 82849

In [19]:

from pinecone import Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [20]:
# 기존 인덱스 정보 확인 (차원수 등)
existing_index = pc.Index("wine-embeddings")
index_stats = existing_index.describe_index_stats()

In [21]:
index_stats

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 129971}},
 'total_vector_count': 129971,
 'vector_type': 'dense'}

In [22]:
query = "달콤한 맛을 느낄 수 있는 와인"

results = vector_store.similarity_search(
    query, 
    k=5, 
    namespace=""
)

results

[Document(id='35025', metadata={}, page_content=": 35025\ncountry: Hungary\ndescription: Orange peel, honey and spice notes linger over a bouquet of hyacinth and freesias. So beautifully structured, it's as if each sip unleashes a wave of sweet amber honey that undulates on the palate amidst flashes of golden marmalade and a vibrant citrus acidity.\ndesignation: Aszú 6 Puttonyos - 500 ml\npoints: 92\nprice: 80.0\nprovince: Tokaj\nregion_1: \nregion_2: \ntaster_name: Anna Lee C. Iijima\ntaster_twitter_handle: \ntitle: Tokaj Classic 1999 Aszú 6 Puttonyos - 500 ml Tokaji (Tokaj)\nvariety: Tokaji\nwinery: Tokaj Classic"),
 Document(id='35791', metadata={}, page_content=": 35791\ncountry: France\ndescription: Liversan makes a serious style of wine from its 125-acre vineyard, situated in the north of the Haut-Médoc. Dense, this is balanced between the black plum fruits and dry tannins. It's a wine for long-term aging, so drink from 2018.\ndesignation: \npoints: 90\nprice: 26.0\nprovince: Bor

In [23]:
index_stats = existing_index.describe_index_stats()
print(index_stats)


{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 129971}},
 'total_vector_count': 129971,
 'vector_type': 'dense'}


In [24]:
# 국가별 필터링
def search_wine_by_country(query, country="US"):
    results = vector_store.similarity_search(
        query,
        k=5,
        filter={"country": country},
        namespace=""
    )
    return results

In [25]:
search_wine_by_country(query)

[]

In [26]:
# recommend_wine

In [27]:
WINE_SOMMELIER_SYSTEM = """
            Persona:

            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

            Role:

            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

            Examples:

            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
        """

In [28]:
def recommend_wine_chain(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", WINE_SOMMELIER_SYSTEM),
        ("human", """
            와인 페이링 추천에 아래 요리/맛, 와인 리뷰만을 참고하여 한글로 답변해 주시기 바랍니다.

            요리/맛:
            {dish_flavor}

            와인 리뷰:
            {wine_reviews}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    
    return chain

# chain 연결

In [29]:

runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine_chain)

chain = runnable_1 | runnable_2 | runnable_3

In [30]:

%pip install -qU grandalf

Note: you may need to restart the kernel to use updated packages.


In [31]:
# 음식이미지 링크 > 음식찾아서 음식맛묘사 
# > 음식맛이 와인평에 있는 와인 검색  
# > 와인평+음식맛 > 소믈리에메시지
chain.get_graph().print_ascii()

+----------------------------------+ 
| describe_dish_flavor_chain_input | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
          +-------------+            
          | search_wine |            
          +-------------+            
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
  +-----------------------------+    
  | recommend_wine_chain_output |    
  +---------

In [32]:
response = chain.invoke(
  {
    "image_url": "https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"
  }
)

In [33]:
response

'요리/맛에 대한 설명이 없으므로, 제공된 와인 리뷰를 바탕으로 와인 페어링을 추천해드리겠습니다. 각 와인의 특징을 고려하여 적절한 요리와의 페어링을 제안합니다.\n\n1. **Szoke Matyás 2011 Pinot Gris (Mátra)**\n   - **특징**: 헤이즐넛과 연기의 감칠맛, 흰 복숭아와 귤의 상큼한 맛, 레몬 껍질의 약간 떫은 마무리.\n   - **페어링 추천**: 이 와인은 해산물 요리와 잘 어울립니다. 특히, 레몬과 허브로 양념한 구운 생선이나 해산물 샐러드와 함께 즐기시면 좋습니다. 와인의 산미가 해산물의 신선함을 돋보이게 할 것입니다.\n\n2. **Szoke Matyás 2011 Irsai Oliver (Mátra)**\n   - **특징**: 흰 꽃과 레몬 껍질의 강렬한 향, 타르트한 귤과 라임의 맛, 꽃과 라임 껍질의 힌트.\n   - **페어링 추천**: 이 와인은 가벼운 아시아 요리, 특히 레몬그라스와 생강을 사용한 태국식 샐러드나 스프링롤과 잘 어울립니다. 와인의 아로마틱한 특성이 요리의 향신료와 조화를 이룰 것입니다.\n\n3. **Tokaj Classic 1999 Aszú 6 Puttonyos (Tokaj)**\n   - **특징**: 오렌지 껍질, 꿀, 향신료, 금빛 마멀레이드, 생동감 있는 시트러스 산미.\n   - **페어링 추천**: 이 달콤한 와인은 블루 치즈나 푸아그라와 같은 진한 맛의 음식과 잘 어울립니다. 또한, 디저트로는 아몬드 타르트나 크렘 브륄레와 함께 즐기시면 좋습니다.\n\n4. **Budimir 2013 Župska Tamjanika (Župa)**\n   - **특징**: 구스베리, 망고, 흰 꽃의 향, 흰 복숭아와 노란 복숭아의 맛.\n   - **페어링 추천**: 이 와인은 과일 샐러드나 가벼운 치즈 플래터와 잘 어울립니다. 특히, 신선한 과일과 함께 즐기면 와인의 과일 향이 더욱 돋보일 것입니다.\n\n5. **Tokaj Kereskedőház 2001 Aszú 4 Puttonyos